In [1]:
import warnings
warnings.filterwarnings('ignore')

In [751]:
import pandas as pd
pd.set_option('display.max_columns', 999)
df_a = pd.read_csv("ERSA.csv")
df_d = pd.read_csv("ERSD.csv")
df_s = pd.read_csv("ERSS.csv")


In [754]:
from datetime import datetime
df_a["admission_date"] = df_a["ADMISSION_STATUS_DATE"].apply(lambda x: datetime.strptime(x, "%d-%b-%y"))

df_admission = df_a[["EMPLID","admission_date"]]
df_admission.shape

(83694, 2)

In [755]:
def QBXEncode(x):
    l = ['QBX']
    code = {"0" : "A", "1" : "B", "2" : "C","3":"T","4":"U","5":"V","6":"X","7":"Y","8":"Z","9":"K" }
    for i, v in enumerate(list(str(x)),start=1):
        if i % 2 ==0:
            l.append(code[str(v)])
        else:
            l.append(str(v))
    return "".join(l)

def get_clean_data(df):
    dfCBA=df[((df['STUDENT_COLLEGE_CODE']=='CBA') | (df['CLASS_COLLEGE_CODE']=='CBA')) & (df['STUDENT_ACAD_PROG_PRIMARY_CODE']=='UGD')]
    dfCBA = dfCBA.sort_values(by=['EMPLID','CLASS_TERM'])

    df_clean =dfCBA[['CTERM_TERM_LDESC','CLASS_TERM','EMPLID','STUDENT_ACAD_CAREER','STUDENT_ACAD_PROG_PRIM_LDESC','STUDENT_COLLEGE_CODE','STUDENT_DEPT_CODE','STUDENT_ACAD_PLAN','STUDENT_SUBACAD_PLAN',
             'CLASS_SID','CLASS_COLLEGE_CODE','CLASS_DEPT','CLASS_SUBDEPT','CLASS_I','CLASS_DESCR','ONLINE_COURSE','ENRL_UNITS_TAKEN','ENRL_GRADING_BASIS','ENRL_OFFICIAL_GRADE',
                  'CLASS_STDN_MTG_PAT','CLASSM_MEETING_TIME_START','CLASSM_MONDAY','CLASSM_TUESDAY',
                  'CLASSM_WEDNESDAY','CLASSM_THURSDAY','CLASSM_FRIDAY','CLASSM_SATURDAY','CLASSM_SUNDAY','STUDENT_TYPE_FINAL','RES_TUITION_RESIDENCY']]
    
    return df_clean

In [756]:
df_dataset_1 =pd.read_csv('dataset-Encrypted-2012-13.csv')
df_dataset_2 =pd.read_csv('dataset2-Encrypted-2012-13.csv')
print(df_dataset_1.shape)
print(df_dataset_2.shape)

df_dataset = pd.concat([df_dataset_1, df_dataset_2])
print(df_dataset.shape)
df1 = get_clean_data(df_dataset)
print(df1.shape)

(256112, 133)
(181396, 133)
(437508, 133)
(49882, 30)


In [757]:
df_encrypted_1=pd.read_csv('encrypted-2009-2012.csv')
df_encrypted_2=pd.read_csv('encrypted-2009-2012-part2.csv')

print(df_encrypted_1.shape)
print(df_encrypted_2.shape)
df_encrypted = pd.concat([df_encrypted_1, df_encrypted_2])
print(df_encrypted.shape)
df2 = get_clean_data(df_encrypted)
print(df2.shape)

(949153, 127)
(31060, 127)
(980213, 127)
(122476, 30)


In [758]:
df_spring = pd.read_csv('Spring2014dataset.csv', encoding="ISO-8859-1")
df = df_spring[((df_spring['STUDENT_COLLEGE_CODE']=='CBA') | (df_spring['CLASS_COLLEGE_CODE']=='CBA')) & (df_spring['STUDENT_ACAD_PROG_PRIMARY_CODE']=='UGD')]
df3 = df.sort_values(by=['EMPLID','CLASS_TERM'])
df3['EMPLID']= df3['EMPLID'].apply(lambda x: '{0:0>9}'.format(x))
def QBXEncode(x):
    l = ['QBX']
    code = {"0" : "A", "1" : "B", "2" : "C","3":"T","4":"U","5":"V","6":"X","7":"Y","8":"Z","9":"K" }
    for i, v in enumerate(list(str(x)),start=1):
        if i % 2 ==0:
            l.append(code[str(v)])
        else:
            l.append(str(v))
    return "".join(l)

#list
df3['EMPLID']=df3['EMPLID'].apply(QBXEncode)
df3=df3[['CTERM_TERM_LDESC','CLASS_TERM','EMPLID','STUDENT_ACAD_CAREER','STUDENT_ACAD_PROG_PRIM_LDESC','STUDENT_COLLEGE_CODE','STUDENT_DEPT_CODE','STUDENT_ACAD_PLAN','STUDENT_SUBACAD_PLAN',
             'CLASS_SID','CLASS_COLLEGE_CODE','CLASS_DEPT','CLASS_SUBDEPT','CLASS_I','CLASS_DESCR','ONLINE_COURSE','ENRL_UNITS_TAKEN','ENRL_GRADING_BASIS','ENRL_OFFICIAL_GRADE',
                  'CLASS_STDN_MTG_PAT','CLASSM_MEETING_TIME_START','CLASSM_MONDAY','CLASSM_TUESDAY',
                  'CLASSM_WEDNESDAY','CLASSM_THURSDAY','CLASSM_FRIDAY','CLASSM_SATURDAY','CLASSM_SUNDAY','STUDENT_TYPE_FINAL','RES_TUITION_RESIDENCY']]
print(df3.shape)

(15895, 30)


In [759]:
df_student_lvl=pd.read_csv('SECTION_STUDENT_LVL_DATA-2017-2014.csv')
df4 = get_clean_data(df_student_lvl)
print(df4.shape)

(113787, 30)


In [761]:
dataList = [df1,df2, df3,df4]
for data in dataList:
    print(data.shape)
dfCBA0 = pd.concat(dataList).sort_values(by=['EMPLID','CLASS_TERM'])
print(dfCBA0.shape)

(49882, 30)
(122476, 30)
(15895, 30)
(113787, 30)
(302040, 30)


In [782]:
dfCBA0["course_date"] = dfCBA0["CTERM_TERM_LDESC"].apply(lambda x: datetime.strptime(x.replace("Fall","09 01").replace("Spring","03 01").replace("Summer","06 01"), "%m %d %Y"))
dfCBA1 = dfCBA0["course_date"].groupby(dfCBA0["EMPLID"]).agg({'course_counts': 'count', 'start_date': np.min, 'end_date': np.max })
dfCBA = dfCBA0.merge(dfCBA1, on = ['EMPLID'], how ='left')
dfCBA.shape

(302040, 34)

In [783]:
import numpy as np

#select from 11 classes
cbaClassList = ["Business Communications","Business Finance","Business Statistics I","Business Strategy & Policy","International Business","Legal & Regulatry Env Business","Management Information Systems","Marketing","Organizational Behavior","Princip of Mgmt and Operations","Principles of Management"]
cbaClassCode = ['HRM 360', 'MKTG 300', 'MGMT 425', 'I S 310', 'MGMT 300', 'FIN 300', 'BLAW 320', 'I S 300', 'CBA 300', 'I S 301']
quantitativeList = ['FIN 300', 'I S 300', 'I S 310']

dfCBA11 =  dfCBA[dfCBA['CLASS_DESCR'].isin(cbaClassList)]

#select only valid scores: ['A', 'B', 'C', 'D', 'F']
scoreDict = {"A" : 4, "B" : 3, "C" : 2, "D": 1, "F": 0}
scoreList = list(scoreDict.keys())
 
df =  dfCBA11[ dfCBA11['ENRL_OFFICIAL_GRADE'].isin(scoreList)]

majorList = ['Accountancy BS', 'Finance BS', 'Human Resources Management BS', 'International Business BS',  'Management BS',  'Marketing BS',  'Mgmt Info Systems BS', 'Oper Supply Chain Mgmt  BS', 'Operations Management BS']
df_ttest = df[df['STUDENT_ACAD_PLAN'].isin(majorList)]

df_ttest["score"] = df_ttest["ENRL_OFFICIAL_GRADE"].map(scoreDict)
df_ttest["lesson"] = df_ttest["CLASS_I"].apply(lambda x: x[:-3])  


df_ttest['units']= df_ttest['ENRL_UNITS_TAKEN'].apply(lambda x: int(x))
df_gpa = df_ttest.groupby('EMPLID').apply(lambda x: np.average(x["score"], weights = x["units"], axis = 0)).to_frame(name = 'GPA').reset_index()
df_ttest = df_ttest.merge(df_gpa, on = ['EMPLID'], how ='left')
print(df_ttest.shape)

df_quantGPA = df_ttest[['EMPLID', 'score', 'lesson']].loc[df_ttest['lesson'].isin(quantitativeList)]
df_quantGPA1 = df_quantGPA.groupby('EMPLID').agg({'score':'mean'}).rename(columns={'score':'quantGPA'}).reset_index()
df_ttest = df_ttest.merge(df_quantGPA1, on = ['EMPLID'], how ='left') 

df_ttest.head()

(79917, 38)


,CTERM_TERM_LDESC,CLASS_TERM,EMPLID,STUDENT_ACAD_CAREER,STUDENT_ACAD_PROG_PRIM_LDESC,STUDENT_COLLEGE_CODE,STUDENT_DEPT_CODE,STUDENT_ACAD_PLAN,STUDENT_SUBACAD_PLAN,CLASS_SID,CLASS_COLLEGE_CODE,CLASS_DEPT,CLASS_SUBDEPT,CLASS_I,CLASS_DESCR,ONLINE_COURSE,ENRL_UNITS_TAKEN,ENRL_GRADING_BASIS,ENRL_OFFICIAL_GRADE,CLASS_STDN_MTG_PAT,CLASSM_MEETING_TIME_START,CLASSM_MONDAY,CLASSM_TUESDAY,CLASSM_WEDNESDAY,CLASSM_THURSDAY,CLASSM_FRIDAY,CLASSM_SATURDAY,CLASSM_SUNDAY,STUDENT_TYPE_FINAL,RES_TUITION_RESIDENCY,course_date,course_counts,start_date,end_date,score,lesson,units,GPA,quantGPA
0,Fall 2009,2094,QBX0A0A1K0V6,Undergraduate,Undergraduate Degree,CBA,FIN,International Business BS,NaN,24209,CBA,Management,Human Resources Management,HRM 360 08,Organizational Behavior,Not_Online,3,GRD,C,MW,15:30:00,Y,N,Y,N,N,N,N,Returning UG,R,2009-09-01,4,2009-09-01,2011-03-01,2,HRM 360,3,2.0,NaN
1,Spring 2011,2112,QBX0A0A1K0V6,Undergraduate,Undergraduate Degree,CBA,FIN,International Business BS,OTHLANGU1,29574,CBA,Marketing,NaN,MKTG 300 02,Marketing,Not_Online,3,GCR,C,M,19:00:00,Y,N,N,N,N,N,N,Continuing UG,R,2011-03-01,4,2009-09-01,2011-03-01,2,MKTG 300,3,2.0,NaN
2,Spring 2009,2092,QBX0A0A2A1T5,Undergraduate,Undergraduate Degree,CBA,FIN,Finance BS,FINAN MGMT,35832,CBA,Management,NaN,MGMT 425 12,Business Strategy & Policy,Not_Online,3,GRD,B,TU,19:00:00,N,Y,N,N,N,N,N,Continuing UG,R,2009-03-01,8,2009-03-01,2010-09-01,3,MGMT 425,3,2.0,1.0
3,Spring 2009,2092,QBX0A0A2A1T5,Undergraduate,Undergraduate Degree,CBA,FIN,Finance BS,FINAN MGMT,36626,CBA,Management,Human Resources Management,HRM 360 08,Organizational Behavior,Not_Online,3,GRD,C,TTH,17:30:00,N,Y,N,Y,N,N,N,Continuing UG,R,2009-03-01,8,2009-03-01,2010-09-01,2,HRM 360,3,2.0,1.0
4,Fall 2009,2094,QBX0A0A2A1T5,Undergraduate,Undergraduate Degree,CBA,FIN,Finance BS,FINAN MGMT,29153,CBA,Information Systems,NaN,I S 310 11,Business Statistics I,Not_Online,3,GRD,D,TH,19:00:00,N,N,N,Y,N,N,N,Continuing UG,R,2009-09-01,8,2009-03-01,2010-09-01,1,I S 310,3,2.0,1.0


# Class score description for Management Information Systems

In [784]:
def describe(className):
    df = df_ttest[ df_ttest['CLASS_DESCR']== className  ]
    return df.groupby(["STUDENT_ACAD_PLAN"])[["score"]].describe()

In [785]:
d = describe("Management Information Systems")
d

score                                          \
                                count      mean       std  min  25%  50%  75%   
STUDENT_ACAD_PLAN                                                               
Accountancy BS                 1781.0  2.824818  0.848443  0.0  2.0  3.0  3.0   
Finance BS                     1265.0  2.732016  0.848562  0.0  2.0  3.0  3.0   
Human Resources Management BS   361.0  2.645429  0.810718  0.0  2.0  3.0  3.0   
International Business BS       449.0  2.652561  0.834255  0.0  2.0  3.0  3.0   
Management BS                  1434.0  2.638773  0.847345  0.0  2.0  3.0  3.0   
Marketing BS                   1401.0  2.659529  0.815298  0.0  2.0  3.0  3.0   
Mgmt Info Systems BS            433.0  2.854503  0.852330  0.0  2.0  3.0  3.0   
Oper Supply Chain Mgmt  BS       85.0  2.858824  0.847306  0.0  2.0  3.0  3.0   
Operations Management BS         11.0  3.090909  0.831209  2.0  2.5  3.0  4.0   

                                    
                               max  
STUDENT_ACAD_PLAN                   
Accountancy BS                 4.0  
Finance BS                     4.0  
Human Resources Management BS  4.0  
International Business BS      4.0  
Management BS                  4.0  
Marketing BS                   4.0  
Mgmt Info Systems BS           4.0  
Oper Supply Chain Mgmt  BS     4.0  
Operations Management BS       4.0

# Hypothesis1 (New Method): Higher score for student in major class

For each major, do TTest on  avg(majorClass) vs avg(nonMajorClass)

• m = 2: HRM HRM 360

• m = 3: MGMT MGMT 300

• m = 4: MKTG MKTG 300

• m = 5: IB CBA 300-International business

• m = 6: IS IS 300; IS 310

• m = 7: FIN FIN 300

In [786]:
import scipy 
import scipy.stats
import numpy as np

majorList.sort()

def run_ttest_on_class_major(classCode, major):
    df = df_ttest[ df_ttest['STUDENT_ACAD_PLAN']== major  ]
    df = df[ df['lesson'].isin(cbaClassCode)]
    
    if major != 'Mgmt Info Systems BS':
        group1 = df[ df['lesson'] == classCode ]
        group2 = df[ df['lesson'] != classCode ]
    else:
        group1 = df[  df['lesson'].isin(['I S 300', 'I S 310'])] 
        group2 = df[ ~df['lesson'].isin(['I S 300', 'I S 310'])] 
    
    return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean(), group1["score"].count(), group2["score"].count())

majorClassList = [('Human Resources Management BS', 'HRM 360'), ('Management BS', 'MGMT 300'), ('Marketing BS', 'MKTG 300'), \
                  ('International Business BS','CBA 300'), ('Mgmt Info Systems BS', 'IS300/IS 310'), ('Finance BS', 'FIN 300')]
     
t_test_results = {}
scores = {}
for  majorName, className in majorClassList:
    t_test_results[majorName + " : " + className], scores[majorName + " : " + className] = run_ttest_on_class_major(className, majorName)

#print(t_test_results)
results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')

results_df['majorClassScore'] = scores_df[0].to_numpy()
results_df['nonMajorClassScore'] = scores_df[1].to_numpy()
results_df['majorClassCount'] = scores_df[2].to_numpy()
results_df['nonMajorClassCount'] = scores_df[3].to_numpy()
results_df

,statistic,pvalue,majorClassScore,nonMajorClassScore,majorClassCount,nonMajorClassCount
Human Resources Management BS : HRM 360,-0.994884,3.198575e-01,2.557292,2.606124,384,3331
Management BS : MGMT 300,-1.176587,2.393776e-01,2.564351,2.590806,1857,14553
Marketing BS : MKTG 300,2.271476,2.313191e-02,2.642953,2.588261,1490,13902
International Business BS : CBA 300,-4.665107,3.162630e-06,2.444444,2.634775,576,4578
Mgmt Info Systems BS : IS300/IS 310,-0.586120,5.578234e-01,2.628889,2.648973,900,3749
Finance BS : FIN 300,6.091207,1.150127e-09,2.851661,2.692491,1355,12572


# Hypothesis1 (Old Method): Higher score for major student


For each course, do TTest on avg(majorStudent take major class Score) vs avg(nonMajorStudent take major class score)

• m = 2: HRM HRM 360

• m = 3: MGMT MGMT 300

• m = 4: MKTG MKTG 300

• m = 5: IB CBA 300-International business

• m = 6: IS IS 300; IS 310

• m = 7: FIN FIN 300

In [787]:
import scipy 
import scipy.stats
import numpy as np

majorList.sort()

def run_ttest_on_class_major(classCode, major):
    df = df_ttest[ df_ttest['lesson']== classCode  ]
    
    if major == 'Mgmt Info Systems BS':
        df = df_ttest[ df_ttest['lesson'].isin(['I S 300', 'I S 310']) ]
    else:
        df = df_ttest[ df_ttest['lesson']== classCode  ]
    group1 = df[ df['STUDENT_ACAD_PLAN'] == major ]
    group2 = df[ df['STUDENT_ACAD_PLAN'] != major ]
    return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean(), group1["score"].count(), group2["score"].count())

majorClassList = [('Human Resources Management BS', 'HRM 360'), ('Management BS', 'MGMT 300'), ('Marketing BS', 'MKTG 300'), \
                  ('International Business BS','CBA 300'), ('Mgmt Info Systems BS', 'IS 300/310'), ('Finance BS', 'FIN 300')]
     
t_test_results = {}
scores = {}
for  majorName, className in majorClassList:
    t_test_results[className + " : " + majorName], scores[className + " : " + majorName ] = run_ttest_on_class_major(className, majorName)

#print(t_test_results)
results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')

results_df['majorScore'] = scores_df[0].to_numpy()
results_df['nonMajorScore'] = scores_df[1].to_numpy()
results_df

,statistic,pvalue,majorScore,nonMajorScore
HRM 360 : Human Resources Management BS,-0.455351,6.488684e-01,2.557292,2.580923
MGMT 300 : Management BS,-4.078622,4.571056e-05,2.564351,2.660422
MKTG 300 : Marketing BS,-0.432440,6.654329e-01,2.642953,2.653528
CBA 300 : International Business BS,-0.701952,4.827293e-01,2.444444,2.473967
IS 300/310 : Mgmt Info Systems BS,1.208981,2.266892e-01,2.628889,2.591015
FIN 300 : Finance BS,5.078887,3.880284e-07,2.851661,2.693636


# Hypothesis 2: Class schedule time on Class Score

## Daytime: before 18:00 vs Night: after 18:00

AllClass: all class combined

In [788]:
df_ttest["hour"] = df_ttest["CLASSM_MEETING_TIME_START"].str.split(":").str[0]
df_ttest['hour'] = df_ttest['hour'].replace(np.nan, 0)

df_ttest["morning"] = df_ttest["hour"].apply(lambda x: 1 if int(x) < 12 else  0)
df_ttest["afternoon"] = df_ttest["hour"].apply(lambda x: 1 if int(x) >= 12 and int(x) < 18 else  0)
df_ttest["night"] = df_ttest["hour"].apply(lambda x: 1 if int(x) >= 18  else  0)

df_ttest["dayhour"] = df_ttest["hour"].apply(lambda x: 1 if int(x) < 18 else  0)
 
cbaClassCode = ['HRM 360', 'MKTG 300', 'MGMT 425', 'MGMT 300', 'FIN 300', 'BLAW 320', 'I S 300', 'CBA 300', 'I S 301']    
    
def run_ttest_class_time(classCode, dayTime):
    if classCode == "AllClass":
        df = df_ttest 
    else:
        df = df_ttest[ df_ttest['lesson']== classCode  ]
    
    group1 = df[ df[dayTime] == 1 ]
    group2 = df[ df[dayTime] == 0 ]
    
    return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean(), group1["score"].count(), group2["score"].count())    

cbaClassCode = ['AllClass', 'HRM 360', 'MKTG 300', 'MGMT 425', 'I S 310', 'MGMT 300', 'FIN 300', 'BLAW 320', 'I S 300', 'CBA 300', 'I S 301']

t_test_results = {}
scores = {}

for classCode in cbaClassCode:
    for dayTime in ["dayhour"]:
        t_test_results[classCode + " : " + dayTime], scores[classCode + " : " + dayTime] = run_ttest_class_time(classCode, dayTime)
    
#print(t_test_results)
results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')

results_df['inTimePeriod'] = scores_df[0].to_numpy()
results_df['outTimePeriod'] = scores_df[1].to_numpy()

results_df['inTimeCount'] = scores_df[2].to_numpy()
results_df['outTimeOut'] = scores_df[3].to_numpy()

results_df

,statistic,pvalue,inTimePeriod,outTimePeriod,inTimeCount,outTimeOut
AllClass : dayhour,1.268250,2.047123e-01,2.670808,2.660967,62137,17780
HRM 360 : dayhour,8.170047,3.538951e-16,2.626759,2.411536,6465,1803
MKTG 300 : dayhour,0.455407,6.488284e-01,2.654195,2.644465,5960,2177
MGMT 425 : dayhour,-0.711404,4.768564e-01,2.857825,2.874572,5866,1459
I S 310 : dayhour,2.635469,8.418765e-03,2.491647,2.428571,5567,2226
MGMT 300 : dayhour,9.952185,3.289083e-23,2.703611,2.495855,5982,2654
FIN 300 : dayhour,-7.820153,5.932659e-15,2.669024,2.880142,6221,1969
BLAW 320 : dayhour,-0.530730,5.956206e-01,2.929708,2.942628,6473,1621
I S 300 : dayhour,-7.627947,2.690978e-14,2.683806,2.869507,5718,1502
CBA 300 : dayhour,-3.087722,2.023695e-03,2.459843,2.562955,7259,961


## Daytime: before 12:00 vs aternoon + night


In [789]:
t_test_results = {}
scores = {}

for classCode in cbaClassCode:
    for dayTime in ["morning"]:
        t_test_results[classCode + " : " + dayTime], scores[classCode + " : " + dayTime] = run_ttest_class_time(classCode, dayTime)
    
#print(t_test_results)
results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')

results_df['inTimePeriod'] = scores_df[0].to_numpy()
results_df['outTimePeriod'] = scores_df[1].to_numpy()

results_df['inTimeCount'] = scores_df[2].to_numpy()
results_df['outTimeOut'] = scores_df[3].to_numpy()

results_df

,statistic,pvalue,inTimePeriod,outTimePeriod,inTimeCount,outTimeOut
AllClass : morning,-0.074680,9.404698e-01,2.668255,2.668778,24350,55567
HRM 360 : morning,1.399460,1.617126e-01,2.603758,2.570066,2395,5873
MKTG 300 : morning,4.085082,4.448408e-05,2.737110,2.634172,1377,6760
MGMT 425 : morning,-3.955604,7.706167e-05,2.802481,2.884682,2096,5229
I S 310 : morning,-1.320553,1.866892e-01,2.454864,2.484498,2858,4935
MGMT 300 : morning,5.420043,6.119152e-08,2.699973,2.594138,3723,4913
FIN 300 : morning,-8.316332,1.053476e-16,2.575854,2.783674,2518,5672
BLAW 320 : morning,-1.379127,1.678939e-01,2.913731,2.942018,2782,5312
I S 300 : morning,4.318696,1.590540e-05,2.801092,2.699156,1649,5571
CBA 300 : morning,2.353575,1.861729e-02,2.514979,2.457087,2103,6117


## Afternoon   vs Morning + Night


In [790]:
t_test_results = {}
scores = {}

for classCode in cbaClassCode:
    for dayTime in ["afternoon"]:
        t_test_results[classCode + " : " + dayTime], scores[classCode + " : " + dayTime] = run_ttest_class_time(classCode, dayTime)
    
#print(t_test_results)
results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')

results_df['inTimePeriod'] = scores_df[0].to_numpy()
results_df['outTimePeriod'] = scores_df[1].to_numpy()

results_df['inTimeCount'] = scores_df[2].to_numpy()
results_df['outTimeOut'] = scores_df[3].to_numpy()

results_df

,statistic,pvalue,inTimePeriod,outTimePeriod,inTimeCount,outTimeOut
AllClass : afternoon,1.125370,2.604358e-01,2.672453,2.665179,37787,42130
HRM 360 : afternoon,5.461330,4.863011e-08,2.640295,2.521201,4070,4198
MKTG 300 : afternoon,-2.679778,7.381929e-03,2.629282,2.680360,4583,3554
MGMT 425 : afternoon,3.006635,2.650528e-03,2.888594,2.832068,3770,3555
I S 310 : afternoon,3.838628,1.247031e-04,2.530454,2.443352,2709,5084
MGMT 300 : afternoon,4.295955,1.758486e-05,2.709606,2.615023,2259,6377
FIN 300 : afternoon,0.988470,3.229515e-01,2.732379,2.709383,3703,4487
BLAW 320 : afternoon,0.888563,3.742645e-01,2.941750,2.924370,3691,4403
I S 300 : afternoon,-9.936211,4.074570e-23,2.636274,2.833704,4069,3151
CBA 300 : afternoon,-4.178149,2.969332e-05,2.437355,2.530026,5156,3064


# Hypothesis 2: (Old Method) Class schedule time on Class Score


In [791]:
df_ttest["hour"] = df_ttest["CLASSM_MEETING_TIME_START"].str.split(":").str[0]
df_ttest['hour'] = df_ttest['hour'].replace(np.nan, 0)

df_ttest["morning"] = df_ttest["hour"].apply(lambda x: 1 if int(x) < 12 else  0)
df_ttest["afternoon"] = df_ttest["hour"].apply(lambda x: 1 if int(x) >= 12 and int(x) < 18 else  0)
df_ttest["night"] = df_ttest["hour"].apply(lambda x: 1 if int(x) >= 18  else  0)
 
cbaClassCode = ['HRM 360', 'MKTG 300', 'MGMT 425', 'I S 310', 'MGMT 300', 'FIN 300', 'BLAW 320', 'I S 300', 'CBA 300', 'I S 301']    
    
def run_ttest_class_time(classCode, dayTime):
    if classCode != "AllClass":
        df = df_ttest[ df_ttest['lesson']== classCode  ]
    else:
        df = df_ttest 
    
    group1 = df[ df[dayTime] == 1 ]
    group2 = df[ df[dayTime] == 0 ]
    
    return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean(), group1["score"].count(), group2["score"].count())    

cbaClassCode = ['AllClass', 'HRM 360', 'MKTG 300', 'MGMT 425', 'I S 310', 'MGMT 300', 'FIN 300', 'BLAW 320', 'I S 300', 'CBA 300', 'I S 301']

t_test_results = {}
scores = {}

for classCode in cbaClassCode:
    for dayTime in ["morning", "afternoon", "night"]:
        t_test_results[classCode + " : " + dayTime], scores[classCode + " : " + dayTime] = run_ttest_class_time(classCode, dayTime)
    
#print(t_test_results)
results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')

results_df['inTimePeriod'] = scores_df[0].to_numpy()
results_df['outTimePeriod'] = scores_df[1].to_numpy()

results_df['inTimeCount'] = scores_df[2].to_numpy()
results_df['outTimeOut'] = scores_df[3].to_numpy()

results_df

,statistic,pvalue,inTimePeriod,outTimePeriod,inTimeCount,outTimeOut
AllClass : morning,-0.074680,9.404698e-01,2.668255,2.668778,24350,55567
AllClass : afternoon,1.125370,2.604358e-01,2.672453,2.665179,37787,42130
AllClass : night,-1.268250,2.047123e-01,2.660967,2.670808,17780,62137
HRM 360 : morning,1.399460,1.617126e-01,2.603758,2.570066,2395,5873
HRM 360 : afternoon,5.461330,4.863011e-08,2.640295,2.521201,4070,4198
HRM 360 : night,-8.170047,3.538951e-16,2.411536,2.626759,1803,6465
MKTG 300 : morning,4.085082,4.448408e-05,2.737110,2.634172,1377,6760
MKTG 300 : afternoon,-2.679778,7.381929e-03,2.629282,2.680360,4583,3554
MKTG 300 : night,-0.455407,6.488284e-01,2.644465,2.654195,2177,5960
MGMT 425 : morning,-3.955604,7.706167e-05,2.802481,2.884682,2096,5229


# Hypothesis 2: Different class time has score difference

In [792]:
df_ttest["hour"] = df_ttest["CLASSM_MEETING_TIME_START"].str.split(":").str[0]
df_ttest['hour'] = df_ttest['hour'].replace(np.nan, 0)

df_ttest["morning"] = df_ttest["hour"].apply(lambda x: 1 if int(x) < 12 else  0)
df_ttest["afternoon"] = df_ttest["hour"].apply(lambda x: 1 if int(x) >= 12 and int(x) < 18 else  0)
df_ttest["night"] = df_ttest["hour"].apply(lambda x: 1 if int(x) >= 18  else  0)

def run_ttest_on_day_time(dayTime):
    group1 = df_ttest[ df_ttest[dayTime] == 1 ]
    group2 = df_ttest[ df_ttest[dayTime] == 0 ]
    
    return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean(),  group1["score"].count(), group2["score"].count() )
 
t_test_results = {}
scores = {}
for dayTime in ["morning", "afternoon", "night"]:
    t_test_results[dayTime], scores[dayTime] = run_ttest_on_day_time(dayTime)

results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')

results_df['inPeriodScore'] = scores_df[0].to_numpy()
results_df['outPeriodScore'] = scores_df[1].to_numpy()
results_df['inTimeCount'] = scores_df[2].to_numpy()
results_df['outTimeCount'] = scores_df[3].to_numpy()
results_df

,statistic,pvalue,inPeriodScore,outPeriodScore,inTimeCount,outTimeCount
morning,-0.07468,0.940470,2.668255,2.668778,24350,55567
afternoon,1.12537,0.260436,2.672453,2.665179,37787,42130
night,-1.26825,0.204712,2.660967,2.670808,17780,62137


# Hypothesis 3: Students take 2 or more quantitative classes in one semester has lower average score (of quant classes taken in the period)



In [793]:
df_ttest["quantitative"] = df_ttest['lesson'].apply(lambda x: 1 if x in quantitativeList else  0)
df1 = df_ttest[ df_ttest['lesson'].isin(quantitativeList) ]
df2 = df1[["EMPLID", 'CTERM_TERM_LDESC', 'quantitative', 'lesson','score']].drop_duplicates().sort_values('EMPLID')

#print(df.tail(100))

df3 = df2.groupby(['EMPLID', 'CTERM_TERM_LDESC'])['quantitative'].sum().reset_index().rename(columns={'quantitative':'sum_quantitative'})
df_hyp3 = df1.merge(df3, on = ['EMPLID','CTERM_TERM_LDESC'], how ='left')

df_hyp3.describe()

,course_counts,score,units,GPA,quantGPA,morning,afternoon,night,dayhour,quantitative,sum_quantitative
count,23203.000000,23203.000000,23203.0,23203.000000,23203.000000,23203.000000,23203.000000,23203.000000,23203.000000,23203.0,23203.000000
mean,24.050381,2.637935,3.0,2.676392,2.637935,0.302763,0.451709,0.245529,0.754471,1.0,1.354954
std,10.606901,0.963548,0.0,0.623039,0.779333,0.459463,0.497673,0.430409,0.430409,0.0,0.538277
min,1.000000,0.000000,3.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.000000
25%,18.000000,2.000000,3.0,2.250000,2.000000,0.000000,0.000000,0.000000,1.000000,1.0,1.000000
50%,22.000000,3.000000,3.0,2.666667,2.666667,0.000000,0.000000,0.000000,1.000000,1.0,1.000000
75%,29.000000,3.000000,3.0,3.111111,3.000000,1.000000,1.000000,0.000000,1.000000,1.0,2.000000
max,82.000000,4.000000,3.0,4.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.0,3.000000


In [794]:
df_ttest["quantitative"] = df_ttest['lesson'].apply(lambda x: 1 if x in quantitativeList else  0)
df = df_ttest[ df_ttest['lesson'].isin(quantitativeList) ]

#print(df.tail(100))

df2 = df.groupby(['EMPLID', 'CTERM_TERM_LDESC'])['quantitative'].sum().reset_index().rename(columns={'quantitative':'sum_quantitative'})
df_hyp3 = df.merge(df2, on = ['EMPLID','CTERM_TERM_LDESC'], how ='left')


def run_ttest_on_quant(df_hyp3):
    group1 = df_hyp3[ df_hyp3["sum_quantitative"] >= 2 ]
    group2 = df_hyp3[ df_hyp3["sum_quantitative"] <  2 ]
    
    return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean(),  group1["score"].count(), group2["score"].count())
 
t_test_results = {}
scores = {}
t_test_results["Hypothesis3"], scores["Hypothesis3"] = run_ttest_on_quant(df_hyp3)
results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')
results_df['MoreQuantClassScore'] = scores_df[0].to_numpy()
results_df['LessQuantClassScore'] = scores_df[1].to_numpy()
results_df['countStudentMoreQuant'] = scores_df[2].to_numpy()
results_df['countStudentLessQuant'] = scores_df[3].to_numpy()
results_df

,statistic,pvalue,MoreQuantClassScore,LessQuantClassScore,countStudentMoreQuant,countStudentLessQuant
Hypothesis3,2.931103,0.003381,2.662606,2.624005,8373,14830


# Hypothesis 3b: Students take 2 or more quantitative classes in one semester has lower average score (of all classes taken in the period)

In [795]:
df_ttest["quantitative"] = df_ttest['lesson'].apply(lambda x: 1 if x in quantitativeList else  0)
df = df_ttest[ df_ttest['lesson'].isin(quantitativeList) ]

#print(df.tail(100))

df2 = df.groupby(['EMPLID', 'CTERM_TERM_LDESC'])['quantitative'].sum().reset_index().rename(columns={'quantitative':'sum_quantitative'})
df_hyp3b = df_ttest.merge(df2, on = ['EMPLID','CTERM_TERM_LDESC'], how ='left')

 
t_test_results = {}
scores = {}
t_test_results["Hypothesis3B"], scores["Hypothesis3B"] = run_ttest_on_quant(df_hyp3b)
results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')
results_df['MoreQuantClassScore'] = scores_df[0].to_numpy()
results_df['LessQuantClassScore'] = scores_df[1].to_numpy()
results_df['countStudentMoreQuant'] = scores_df[2].to_numpy()
results_df['countStudentLessQuant'] = scores_df[3].to_numpy()
results_df

,statistic,pvalue,MoreQuantClassScore,LessQuantClassScore,countStudentMoreQuant,countStudentLessQuant
Hypothesis3B,1.709384,0.087386,2.684215,2.668107,13145,34261


# Hypothesis 4: Which Major has higher quantitive class GPA?

In [796]:
df_hyp4 = df_ttest[['EMPLID', 'quantGPA', 'STUDENT_ACAD_PLAN']].loc[df_ttest["quantGPA"].notnull()]
df_hyp4 = df_hyp4.drop_duplicates() 
print(df_hyp4.shape)

df_hyp4["major_mis"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Mgmt Info Systems BS' else  0) 
df_hyp4["major_fin"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Finance BS' else  0) 
df_hyp4["major_acc"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Accountancy BS' else  0) 
df_hyp4["major_mrk"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Marketing BS' else  0) 
df_hyp4["major_mgmt"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Management BS' else  0) 
df_hyp4["major_hrm"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Human Resources Management BS' else  0) 
df_hyp4["major_oscm"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'Oper Supply Chain Mgmt  BS' or x == 'Operations Management BS' else  0) 
df_hyp4["major_ib"] = df_hyp4["STUDENT_ACAD_PLAN"].apply(lambda x: 1 if x == 'International Business BS' else  0) 

def run_ttest_on_major(major):
    group1 = df_hyp4[ df_hyp4[major] == 1 ]
    group2 = df_hyp4[ df_hyp4[major] == 0 ]
    
    return scipy.stats.ttest_ind(group1['quantGPA'], group2['quantGPA']), (group1["quantGPA"].mean(), group2["quantGPA"].mean())
 
t_test_results = {}
scores = {}
for major in ["major_mis", "major_fin", "major_acc", "major_mrk", "major_mgmt", "major_hrm", "major_oscm", "major_ib"]:
    t_test_results[major], scores[major] = run_ttest_on_major(major)

results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')

results_df['majorQuantGPA'] = scores_df[0].to_numpy()
results_df['otherMajorQuantGPA'] = scores_df[1].to_numpy()
results_df


(11470, 3)


,statistic,pvalue,majorQuantGPA,otherMajorQuantGPA
major_mis,1.070605,2.843696e-01,2.665381,2.630608
major_fin,5.999023,2.044978e-09,2.730066,2.611162
major_acc,11.853578,3.179155e-32,2.791055,2.581519
major_mrk,-8.822609,1.279671e-18,2.493207,2.665004
major_mgmt,-7.041867,2.005230e-12,2.525329,2.659497
major_hrm,-3.951921,7.799116e-05,2.501813,2.639607
major_oscm,-0.003002,9.976051e-01,2.632466,2.632654
major_ib,-2.078504,3.768516e-02,2.571176,2.636751


# Hypothesis 5: Students in junior year taking quantitative class have lower score, using addmision date as first day

In [855]:
from datetime import timedelta
    
df_5 = df_ttest.merge(df_admission, on = ['EMPLID'], how ='left')
 
#print(df_ttest.shape)
#df_5["course_date"] = df_5["CTERM_TERM_LDESC"].apply(lambda x: datetime.strptime(x.replace("Fall","09 01").replace("Spring","01 01"), "%m %d %Y"))
df_5["start_time"]  = df_5.course_date - df_5.admission_date
df_5["junior_year"] = df_5["start_time"].apply(lambda x: 1 if x <  timedelta(days=365*3) else 0)

df_5 = df_5.loc[df_5["admission_date"].notnull()]
df_5 = df_5.loc[df_5["quantitative"] == 1]

#hist = df_5["junior_year"].hist(bins=10)

#print(df_5[["CTERM_TERM_LDESC", "course_date", "admission_date", "start_time", "junior_year", "quantitative"]])

def run_ttest_on_h5(df):
    print(df.shape)
    group1 = df[ df["junior_year"] == 1 ]
    group2 = df[ df["junior_year"] == 0 ]
    return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean())

def run_hypothesis5(lesson):
    t_test_results = {}
    scores = {}
    if lesson == "Overall":
        t_test_results["Hypothesis5"], scores["Hypothesis5"] = run_ttest_on_h5(df_5)
    else:
        t_test_results["Hypothesis5"], scores["Hypothesis5"] = run_ttest_on_h5(df_5.loc[df_5["lesson"] == lesson])
 
    results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
    scores_df = pd.DataFrame.from_dict(scores, orient='Index')
    results_df['Junior'] = scores_df[0].to_numpy()
    results_df['Senior'] = scores_df[1].to_numpy()
    results_df['Class Name'] = lesson
    return results_df
    
df_hp5_gpa = run_hypothesis5("Overall")
df_hp5_fin300 = run_hypothesis5("FIN 300")
df_hp5_is300 = run_hypothesis5("I S 300")
df_hp5_is310 = run_hypothesis5("I S 310")

df_dataset = pd.concat([df_hp5_gpa, df_hp5_fin300, df_hp5_is300, df_hp5_is310])
df_dataset


(5258, 50)
(1830, 50)
(1673, 50)
(1755, 50)


,statistic,pvalue,Junior,Senior,Class Name
Hypothesis5,4.636618,0.000004,2.730633,2.606762,Overall
Hypothesis5,3.918248,0.000092,2.777433,2.590278,FIN 300
Hypothesis5,-1.484103,0.137970,2.707566,2.773649,I S 300
Hypothesis5,4.264871,0.000021,2.665385,2.458300,I S 310


# Hypothesis 5 : (New) using first class date as first day.  Students in junior year taking quantitative class have lower score

In [854]:
from datetime import timedelta

df_hyp5 = df_ttest

df_hyp5["course_start_time"]  = df_hyp5.course_date - df_hyp5.start_date
df_hyp5['study_length'] = df_hyp5.end_date - df_hyp5.start_date 
df_hyp5["junior_year"] = df_hyp5["course_start_time"].apply(lambda x: 1 if x <= timedelta(days=365*2) else 0)

df_hyp51 = df_hyp5.loc[(df_hyp5['study_length'] >= timedelta(days=365*2)) & (df_hyp5['course_counts'] >=2)]
df_hyp51 = df_hyp51.loc[df_hyp51["quantitative"] == 1]

#hist = df_hyp51["course_counts"].hist(bins=10)
#print(df_hyp51.shape)

def run_ttest_on_h5(df):
    print(df.shape)
    group1 = df[ df["junior_year"] == 1 ]
    group2 = df[ df["junior_year"] == 0 ]
    
    return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean())

def run_hypothesis5(lesson):
    t_test_results = {}
    scores = {}
    if lesson == "Overall":
        t_test_results["Hypothesis5"], scores["Hypothesis5"] = run_ttest_on_h5(df_hyp51)
    else:
        t_test_results["Hypothesis5"], scores["Hypothesis5"] = run_ttest_on_h5(df_hyp51.loc[df_hyp51["lesson"] == lesson])
 
    results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
    scores_df = pd.DataFrame.from_dict(scores, orient='Index')
    results_df['Junior'] = scores_df[0].to_numpy()
    results_df['Senior'] = scores_df[1].to_numpy()
    results_df['Class Name'] = lesson
    return results_df
    
df_hp5_gpa = run_hypothesis5("Overall")
df_hp5_fin300 = run_hypothesis5("FIN 300")
df_hp5_is300 = run_hypothesis5("I S 300")
df_hp5_is310 = run_hypothesis5("I S 310")

df_dataset = pd.concat([df_hp5_gpa, df_hp5_fin300, df_hp5_is300, df_hp5_is310])
df_dataset

(15662, 48)
(5417, 48)
(4972, 48)
(5273, 48)


,statistic,pvalue,Junior,Senior,Class Name
Hypothesis5,6.183366,6.430559e-10,2.645772,2.546494,Overall
Hypothesis5,6.051256,1.533664e-09,2.734470,2.541303,FIN 300
Hypothesis5,-0.895773,3.704175e-01,2.698012,2.719911,I S 300
Hypothesis5,2.981601,2.880565e-03,2.482353,2.403195,I S 310


In [803]:
from datetime import timedelta

df_50["course_date"] = df_ttest["CTERM_TERM_LDESC"].apply(lambda x: datetime.strptime(x.replace("Fall","09 01").replace("Spring","01 01"), "%m %d %Y"))
df_5["start_time"]  = df_5.course_date - df_5.admission_date
df_5["junior_year"] = df_5["start_time"].apply(lambda x: 1 if x >  timedelta(days=365*2) else 0)

df_5 = df_5.loc[df_5["admission_date"].notnull()]
df_5 = df_5.loc[df_5["quantitative"] == 1]

#print(df_5[["CTERM_TERM_LDESC", "course_date", "admission_date", "start_time", "junior_year", "quantitative"]])

def run_ttest_on_h5(df, check_gpa = False):
    group1 = df[ df["junior_year"] == 1 ]
    group2 = df[ df["junior_year"] == 0 ]
    
    if check_gpa:
        return scipy.stats.ttest_ind(group1['GPA'], group2['GPA']), (group1["GPA"].mean(), group2["GPA"].mean())
    else:
        return scipy.stats.ttest_ind(group1['score'], group2['score']), (group1["score"].mean(), group2["score"].mean())
    
    
t_test_results = {}
scores = {}
t_test_results["Hypothesis5"], scores["Hypothesis5"] = run_ttest_on_h5(df_5)
results_gpa = pd.DataFrame.from_dict(t_test_results, orient='Index')
scores_df = pd.DataFrame.from_dict(scores, orient='Index')
results_gpa['Junior'] = scores_df[0].to_numpy()
results_gpa['Senior'] = scores_df[1].to_numpy()

def run_hypothesis5(lesson):
    t_test_results = {}
    scores = {}
    if lesson == "GPA":
        t_test_results["Hypothesis5"], scores["Hypothesis5"] = run_ttest_on_h5(df_5, True)
    else:
        t_test_results["Hypothesis5"], scores["Hypothesis5"] = run_ttest_on_h5(df_5.loc[df_5["lesson"] == lesson])
 
    results_df = pd.DataFrame.from_dict(t_test_results, orient='Index')
    scores_df = pd.DataFrame.from_dict(scores, orient='Index')
    results_df['Junior'] = scores_df[0].to_numpy()
    results_df['Senior'] = scores_df[1].to_numpy()
    results_df['Class Name'] = lesson
    return results_df
    
df_hp5_gpa = run_hypothesis5("GPA")
df_hp5_fin300 = run_hypothesis5("FIN 300")
df_hp5_is300 = run_hypothesis5("I S 300")
df_hp5_is310 = run_hypothesis5("I S 310")

df_dataset = pd.concat([df_hp5_gpa, df_hp5_fin300, df_hp5_is300, df_hp5_is310])
df_dataset

,statistic,pvalue,Junior,Senior,Class Name
Hypothesis5,-8.169706,3.838236e-16,2.703250,2.993395,GPA
Hypothesis5,-3.696094,2.253922e-04,2.665299,3.016129,FIN 300
Hypothesis5,0.157474,8.748906e-01,2.754975,2.738462,I S 300
Hypothesis5,-5.426744,6.539379e-08,2.489709,3.000000,I S 310


In [800]:
df_ttest.head()

,CTERM_TERM_LDESC,CLASS_TERM,EMPLID,STUDENT_ACAD_CAREER,STUDENT_ACAD_PROG_PRIM_LDESC,STUDENT_COLLEGE_CODE,STUDENT_DEPT_CODE,STUDENT_ACAD_PLAN,STUDENT_SUBACAD_PLAN,CLASS_SID,CLASS_COLLEGE_CODE,CLASS_DEPT,CLASS_SUBDEPT,CLASS_I,CLASS_DESCR,ONLINE_COURSE,ENRL_UNITS_TAKEN,ENRL_GRADING_BASIS,ENRL_OFFICIAL_GRADE,CLASS_STDN_MTG_PAT,CLASSM_MEETING_TIME_START,CLASSM_MONDAY,CLASSM_TUESDAY,CLASSM_WEDNESDAY,CLASSM_THURSDAY,CLASSM_FRIDAY,CLASSM_SATURDAY,CLASSM_SUNDAY,STUDENT_TYPE_FINAL,RES_TUITION_RESIDENCY,course_date,course_counts,start_date,end_date,score,lesson,units,GPA,quantGPA,hour,morning,afternoon,night,dayhour,quantitative,course_start_time,study_length,junior_year
0,Fall 2009,2094,QBX0A0A1K0V6,Undergraduate,Undergraduate Degree,CBA,FIN,International Business BS,NaN,24209,CBA,Management,Human Resources Management,HRM 360 08,Organizational Behavior,Not_Online,3,GRD,C,MW,15:30:00,Y,N,Y,N,N,N,N,Returning UG,R,2009-09-01,4,2009-09-01,2011-03-01,2,HRM 360,3,2.0,NaN,15,0,1,0,1,0,0 days,546 days,0
1,Spring 2011,2112,QBX0A0A1K0V6,Undergraduate,Undergraduate Degree,CBA,FIN,International Business BS,OTHLANGU1,29574,CBA,Marketing,NaN,MKTG 300 02,Marketing,Not_Online,3,GCR,C,M,19:00:00,Y,N,N,N,N,N,N,Continuing UG,R,2011-03-01,4,2009-09-01,2011-03-01,2,MKTG 300,3,2.0,NaN,19,0,0,1,0,0,546 days,546 days,1
2,Spring 2009,2092,QBX0A0A2A1T5,Undergraduate,Undergraduate Degree,CBA,FIN,Finance BS,FINAN MGMT,35832,CBA,Management,NaN,MGMT 425 12,Business Strategy & Policy,Not_Online,3,GRD,B,TU,19:00:00,N,Y,N,N,N,N,N,Continuing UG,R,2009-03-01,8,2009-03-01,2010-09-01,3,MGMT 425,3,2.0,1.0,19,0,0,1,0,0,0 days,549 days,0
3,Spring 2009,2092,QBX0A0A2A1T5,Undergraduate,Undergraduate Degree,CBA,FIN,Finance BS,FINAN MGMT,36626,CBA,Management,Human Resources Management,HRM 360 08,Organizational Behavior,Not_Online,3,GRD,C,TTH,17:30:00,N,Y,N,Y,N,N,N,Continuing UG,R,2009-03-01,8,2009-03-01,2010-09-01,2,HRM 360,3,2.0,1.0,17,0,1,0,1,0,0 days,549 days,0
4,Fall 2009,2094,QBX0A0A2A1T5,Undergraduate,Undergraduate Degree,CBA,FIN,Finance BS,FINAN MGMT,29153,CBA,Information Systems,NaN,I S 310 11,Business Statistics I,Not_Online,3,GRD,D,TH,19:00:00,N,N,N,Y,N,N,N,Continuing UG,R,2009-09-01,8,2009-03-01,2010-09-01,1,I S 310,3,2.0,1.0,19,0,0,1,0,1,184 days,549 days,0


# Unsupvised Learning: K means Clusting on GPA 

In [687]:
from sklearn.cluster import KMeans

df_cluster1 = df_ttest[["EMPLID", "GPA"]][df_ttest["GPA"].notnull()].drop_duplicates().sort_values('GPA')
print(df_cluster1.shape)
# Cluster students based on their quantitive class GPAs into 5 different groups 
X = np.asarray(df_cluster1["GPA"])
kmeans = KMeans(n_clusters=5, random_state=0).fit(X.reshape(-1,1))  

kmeans.cluster_centers_.sum(axis=1)
idx = np.argsort(kmeans.cluster_centers_.sum(axis=1))
lut = np.zeros_like(idx)
lut[idx] = np.arange(5)

#Save the labels
df_cluster1.loc[:,'GPA_cluster'] = lut[kmeans.labels_]
 
df_cluster1.tail()
#df_cluster1[["GPA"]].groupby(df_cluster["GPA_cluster"]).agg(['mean', 'count'])

(11962, 2)


,EMPLID,GPA,GPA_cluster
20728,QBX0A6Z9X5T7,4.0,4
10257,QBX0A5X1T8Y9,4.0,4
2581,QBX0A4B2Z0A5,4.0,4
53456,QBX0B0Z0K0U3,4.0,4
8919,QBX0A5U4A3X8,4.0,4


# Unsupvised Learning: K means Clusting on quant GPA 

In [688]:
from sklearn.cluster import KMeans

df_cluster2 = df_ttest[["EMPLID", "quantGPA"]][df_ttest["quantGPA"].notnull()].drop_duplicates().sort_values('quantGPA')
print(df_cluster2.shape)
# Cluster students based on their quantitive class GPAs into 5 different groups 
X = np.asarray(df_cluster2["quantGPA"])
kmeans = KMeans(n_clusters=5, random_state=0).fit(X.reshape(-1,1))  

kmeans.cluster_centers_.sum(axis=1)
idx = np.argsort(kmeans.cluster_centers_.sum(axis=1))
lut = np.zeros_like(idx)
lut[idx] = np.arange(5)

#Save the labels
df_cluster2.loc[:,'quantGPA_cluster'] = lut[kmeans.labels_]
 
df_cluster2.tail()    
#df_cluster2[["quantGPA"]].groupby(df_cluster["quantGPA_cluster"]).agg(['mean', 'count'])


(9957, 2)


,EMPLID,quantGPA,quantGPA_cluster
3640,QBX0A4U6B1K5,4.0,4
15859,QBX0A6T1Z1X7,4.0,4
53171,QBX0B0Y7U1V1,4.0,4
15962,QBX0A6T3X0X8,4.0,4
29367,QBX0A7Z3X6T2,4.0,4


# Percent of Students in different major by GPA Cluster (from low to high)

In [689]:
df_cluster3 = df_hyp4.merge(df_cluster1, on = ['EMPLID'], how ='left')
majorList = ['major_mis','major_fin','major_acc','major_mrk','major_mgmt','major_hrm','major_oscm','major_ib']

df_cluster3[majorList].groupby(df_cluster3["GPA_cluster"]).agg(['mean'])

,major_mis,major_fin,major_acc,major_mrk,major_mgmt,major_hrm,major_oscm,major_ib
,mean,mean,mean,mean,mean,mean,mean,mean
GPA_cluster,,,,,,,,
0,0.069079,0.157895,0.240132,0.187500,0.220395,0.062500,0.003289,0.059211
1,0.063700,0.170023,0.204684,0.204215,0.222014,0.056674,0.011710,0.066979
2,0.058115,0.176219,0.203535,0.217461,0.212641,0.050348,0.017943,0.063739
3,0.054985,0.185504,0.267981,0.178839,0.187448,0.050819,0.014440,0.059983
4,0.060142,0.198113,0.332547,0.124410,0.168042,0.040094,0.016509,0.060142


# Percent of Students in different major by quant GPA Cluster (from low to high)

In [690]:
df_cluster4 = df_hyp4.merge(df_cluster2, on = ['EMPLID'], how ='left')
majorList = ['major_mis','major_fin','major_acc','major_mrk','major_mgmt','major_hrm','major_oscm','major_ib']

df_cluster4[majorList].groupby(df_cluster4["quantGPA_cluster"]).agg(['mean'])

,major_mis,major_fin,major_acc,major_mrk,major_mgmt,major_hrm,major_oscm,major_ib
,mean,mean,mean,mean,mean,mean,mean,mean
quantGPA_cluster,,,,,,,,
0,0.061058,0.143826,0.199457,0.211669,0.234735,0.055631,0.014925,0.078697
1,0.055708,0.160591,0.198762,0.214237,0.226960,0.066025,0.014099,0.063618
2,0.056121,0.173258,0.196610,0.225612,0.214313,0.045574,0.020339,0.068173
3,0.060948,0.192519,0.271525,0.172203,0.186069,0.049661,0.011932,0.055144
4,0.062349,0.214113,0.343161,0.119865,0.152731,0.034316,0.014500,0.058966


# Time of class by GPA Cluster

In [691]:
df_cluster5 = df_ttest.merge(df_cluster1[["EMPLID","GPA_cluster"]], on = ['EMPLID'], how ='left')

df_cluster5.head()
 
timeList = ['morning','afternoon','night']
df_cluster5[timeList].groupby(df_cluster5["GPA_cluster"]).agg(['mean','count'])
 

morning        afternoon            night       
                 mean  count      mean  count      mean  count
GPA_cluster                                                   
0            0.260777   1879  0.445982   1879  0.293241   1879
1            0.299021  15009  0.464521  15009  0.236458  15009
2            0.304574  26102  0.472148  26102  0.223278  26102
3            0.305848  25104  0.479007  25104  0.215145  25104
4            0.316671  11823  0.476021  11823  0.207308  11823

# Time of class by quantGPA Cluster

In [692]:
df_cluster6 = df_ttest.merge(df_cluster2[["EMPLID","quantGPA_cluster"]], on = ['EMPLID'], how ='left')

df_cluster6.head()
 
timeList = ['morning','afternoon','night']
df_cluster6[timeList].groupby(df_cluster6["quantGPA_cluster"]).agg(['mean','count'])

morning        afternoon            night       
                      mean  count      mean  count      mean  count
quantGPA_cluster                                                   
0.0               0.295696   4136  0.463250   4136  0.241054   4136
1.0               0.304496  18437  0.470521  18437  0.224982  18437
2.0               0.313774  19479  0.478361  19479  0.207865  19479
3.0               0.305090  20099  0.476939  20099  0.217971  20099
4.0               0.300137  13174  0.473964  13174  0.225899  13174

# Maximum number of quant classes per semaster by   GPA Cluster

In [693]:

# For each student, calculate the maximum amount of quantitive classes in a semaster
df_temp = df_hyp3[["EMPLID", "CTERM_TERM_LDESC", "sum_quantitative"]].drop_duplicates()
df_temp1 = df_temp.groupby('EMPLID').agg({'sum_quantitative':'max'}).rename(columns={'sum_quantitative':'maxQuantClass'}).reset_index()

df_q1 = df_temp1.merge(df_cluster1[["EMPLID","GPA_cluster"]], on = ['EMPLID'], how ='left')

 
# For each cluster, count number of students with 
df_q1[["maxQuantClass"]].groupby(df_q1["GPA_cluster"]).agg(['mean', 'count'])

maxQuantClass      
                     mean count
GPA_cluster                    
0                1.214286   280
1                1.378510  1852
2                1.457321  3210
3                1.430591  3112
4                1.398536  1503

# Maximum number of quant classes per semaster by  quant GPA Cluster

In [694]:
# For each student, calculate the maximum amount of quantitive classes in a semaster
df_temp = df_hyp3[["EMPLID", "CTERM_TERM_LDESC", "sum_quantitative"]].drop_duplicates()
df_temp1 = df_temp.groupby('EMPLID').agg({'sum_quantitative':'max'}).rename(columns={'sum_quantitative':'maxQuantClass'}).reset_index()

df_q2 = df_temp1.merge(df_cluster2[["EMPLID","quantGPA_cluster"]], on = ['EMPLID'], how ='left')

 
# For each cluster, count number of students with 
df_q2[["maxQuantClass"]].groupby(df_q2["quantGPA_cluster"]).agg(['mean', 'count'])

maxQuantClass      
                          mean count
quantGPA_cluster                    
0                     1.262270   652
1                     1.361920  2542
2                     1.589755  2245
3                     1.402008  2689
4                     1.367414  1829